# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [10]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [11]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [12]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [13]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from ./barcode_runs_to_upload.csv
Here are the first few entries in the Illumina runs:


,library,sample,sample_type,sort_bin,concentration,date,number_cells,R1
0,pool1,TiteSeq_01_bin1,TiteSeq,1,1.0,240125,384974,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b1_R1_001.fastq.gz; /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b1_R1_001_2.fastq.gz
1,pool1,TiteSeq_01_bin2,TiteSeq,2,1.0,240125,860724,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b2_R1_001.fastq.gz; /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b2_R1_001_2.fastq.gz
2,pool1,TiteSeq_01_bin3,TiteSeq,3,1.0,240125,2596538,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b3_R1_001.fastq.gz; /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b3_R1_001_2.fastq.gz
3,pool1,TiteSeq_01_bin4,TiteSeq,4,1.0,240125,6350060,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b4_R1_001.fastq.gz; /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b4_R1_001_2.fastq.gz
4,pool1,TiteSeq_02_bin1,TiteSeq,1,2.0,240125,630205,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s2-b1_R1_001.fastq.gz; /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s2-b1_R1_001_2.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [24]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'Illumina barcode sequencing for ' + x['library'] + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='NextSeq 2000',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f.strip()) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN42557522,pool1_TiteSeq_01_bin1,Illumina barcode sequencing for pool1TiteSeq_01_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b1_R1_001.fastq.gz,240125-exp1-s1-b1_R1_001.fastq.gz
1,SAMN42557522,pool1_TiteSeq_01_bin1,Illumina barcode sequencing for pool1TiteSeq_01_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b1_R1_001_2.fastq.gz,240125-exp1-s1-b1_R1_001_2.fastq.gz
2,SAMN42557522,pool1_TiteSeq_01_bin2,Illumina barcode sequencing for pool1TiteSeq_01_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b2_R1_001.fastq.gz,240125-exp1-s1-b2_R1_001.fastq.gz
3,SAMN42557522,pool1_TiteSeq_01_bin2,Illumina barcode sequencing for pool1TiteSeq_01_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240410_Illumina-v2_OmiBA286-EG5-FLip/rename/240125-exp1-s1-b2_R1_001_2.fastq.gz,240125-exp1-s1-b2_R1_001_2.fastq.gz
4,SAMN42557522,pool1_TiteSeq_01_bin3,Illumina barcode sequencing for pool1TiteSeq_01_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2024/240313_Illumina_OmiBA286-EG5-FLip_animal-polish_sarbeco-combo-polish/240125-exp1-s1-b3_R1_001.fastq.gz,240125-exp1-s1-b3_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [25]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN42557522,pool2_SortSeq_bin1,6
1,SAMN42557522,pool2_SortSeq_bin4,6
2,SAMN42557522,pool2_SortSeq_bin3,6
3,SAMN42557522,pool2_SortSeq_bin2,6
4,SAMN42557522,pool1_SortSeq_bin2,3
...,...,...,...
75,SAMN42557522,pool1_TiteSeq_07_bin3,2
76,SAMN42557522,pool1_TiteSeq_07_bin2,2
77,SAMN42557522,pool1_TiteSeq_07_bin1,2
78,SAMN42557522,pool1_TiteSeq_06_bin4,2


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [26]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3,filename4,filename5,filename6
0,SAMN42557522,pool1_TiteSeq_01_bin1,Illumina barcode sequencing for pool1TiteSeq_01_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,240125-exp1-s1-b1_R1_001.fastq.gz,240125-exp1-s1-b1_R1_001_2.fastq.gz,NaN,NaN,NaN,NaN
1,SAMN42557522,pool1_TiteSeq_01_bin2,Illumina barcode sequencing for pool1TiteSeq_01_bin2,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,240125-exp1-s1-b2_R1_001.fastq.gz,240125-exp1-s1-b2_R1_001_2.fastq.gz,NaN,NaN,NaN,NaN
2,SAMN42557522,pool1_TiteSeq_01_bin3,Illumina barcode sequencing for pool1TiteSeq_01_bin3,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,240125-exp1-s1-b3_R1_001.fastq.gz,240125-exp1-s1-b3_R1_001_2.fastq.gz,NaN,NaN,NaN,NaN
3,SAMN42557522,pool1_TiteSeq_01_bin4,Illumina barcode sequencing for pool1TiteSeq_01_bin4,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,240125-exp1-s1-b4_R1_001.fastq.gz,240125-exp1-s1-b4_R1_001_2.fastq.gz,NaN,NaN,NaN,NaN
4,SAMN42557522,pool1_TiteSeq_02_bin1,Illumina barcode sequencing for pool1TiteSeq_02_bin1,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,NextSeq 2000,PCR of barcodes from RBD variants,fastq,240125-exp1-s2-b1_R1_001.fastq.gz,240125-exp1-s2-b1_R1_001_2.fastq.gz,NaN,NaN,NaN,NaN


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [27]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [28]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
